In [1]:
using LinearAlgebra
using Flux

In [2]:
input = 10
middle = 20
output = 8

8

In [21]:
Tuple(i for i in 1:5)

(1, 2, 3, 4, 5)

In [20]:
model = Chain(Dense(input, middle), [Chain(Parallel(+, Chain(BatchNorm(middle), Dense(middle, middle, relu)),Dense(middle, middle, relu)), identity) for i in 1:3], Flux.flatten ,Dense(middle, output, tanh))

Chain(
  Dense(10 => 20),                      # 220 parameters
  Array(
    Chain(
      Parallel(
        +,
        Chain(
          BatchNorm(20),                # 40 parameters, plus 40
          Dense(20 => 20, relu),        # 420 parameters
        ),
        Dense(20 => 20, relu),          # 420 parameters
      ),
      identity,
    ),
    Chain(
      Parallel(
        +,
        Chain(
          BatchNorm(20),                # 40 parameters, plus 40
          Dense(20 => 20, relu),        # 420 parameters
        ),
        Dense(20 => 20, relu),          # 420 parameters
      ),
      identity,
    ),
    Chain(
      Parallel(
        +,
        Chain(
          BatchNorm(20),                # 40 parameters, plus 40
          Dense(20 => 20, relu),        # 420 parameters
        ),
        Dense(20 => 20, relu),          # 420 parameters
      ),
      identity,
    ),
  ),
  Flux.flatten,
  Dense(20 => 8, tanh),                 # 168 parameters
)         # Total: 22 tr